# Requests et BeautifulSoup

## Installation Des Packages

In [1]:
# Si nous n'avez pas encore installé ces packages il faut enlever les les lignes suivantes en commentaire etles executer
# !pip install requests --upgrade --quiet
# !pip install beautifulsoup4 --upgrade --quiet
# !pip install pandas --quiet

## Importation des Packages

In [2]:
import requests
from random import randint
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd

## Comment sa fonctionne

La première étape consiste à utiliser le package <code>**Requests**</code> pour télécharger le code HTML de notre page Web (à l'aide de la fonction <code>**requests.get**</code>) qui donne un objet de réponse. Nous pouvons voir si nous pouvons télécharger la page Web ou non en vérifiant la réponse. Statut de notre objet. (Il devrait être compris entre <code>**200 et 299**</code>). Ensuite, nous allons convertir cet objet de réponse en un objet <code>**BeautifulSoup**</code> en utilisant le constructeur <code>**BeautifulSoup()**</code>. Nous utiliserons cet objet pour inspecter notre document et extraire les données souhaitées. <code>**BeautifulSoup**</code> nécessite un argument supplémentaire appelé <code>**parser**</code>. (En bref, **BeautifulSoup** peut également être utilisé pour extraire des informations d'autres langages de balisage. Le **parser** par défaut est <code>**"html.parser"**</code>). Trouvez ci-dessous une fonction d'exemple indépendante qui effectuera la tâche ci-dessus: 
* de téléchargement des pages, 
* de vérification des réponses
d'analyse à l'aide de <code>**BeautifulSoup()**</code>.

In [6]:
def get_item_page(items_url):
    # télécharger la page
    response = requests.get(items_url)
    # vérifier le succès de réponse
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(items_url))
    # Parser la réponse à l'aide de beaufifulSoup
    doc = BeautifulSoup(response.text, 'html.parser')
    return doc

## Scraper la liste des articles de notre site Web cible
Dans notre cas, nous allons scraper https://www.alibaba.com/trade/search?fsb=y&IndexArea=product_en&CatId=&SearchText=Inflight+Items&viewtype=G&tab={page} Nous obtiendrons une liste des produits en vol. Nous obtiendrons un **titre d'article principal**, un **prix**, **des notes** et une **URL** de page pour chaque article. Pour **chaque article**, nous obtiendrons plus de détails sur le produit à partir de la page de l'article (si nécessaire). Nous allons créer un dataframe de Pandas et un fichier **Json** avec les détails de l'élément. Pour chaque article, nous créerons également un fichier **CSV** au format indicatif suivant:

nom, prix, URL
Article JUNIO Fleur Préservée Avec Boîtes, 5.50 - 6.50, https://www.alibaba.com/product-detail/Item-JUNIO-Preserved-Flower-With-Boxes_62339028105.html?s=p

In [7]:
url = "https://www.alibaba.com/trade/search?fsb=y&IndexArea=product_en&CatId=&SearchText=Inflight+Items&viewtype=G&tab=%7Bpage%7D"

In [13]:
doc = get_item_page(url)

## Comprendre les bases de BeautifulSoup

**BeautifulSoup** a plusieurs méthodes pour extraire des informations de notre document analysé. La façon la plus simple de rechercher une balise dans notre document est d'appeler directement le nom de la balise que nous recherchons. Par exemple, si je veux la balise &lt;title&gt;, j'appellerai <code>doc.title</code>. Si je passe <code>doc.a</code>, il renverra l'occurrence de la première balise &lt;a&gt; présente dans mon document. Et si je voulais toutes les balises &lt;a&gt; ? Nous devons utiliser : « La méthode <code>find_all()</code> ».

## BeautifulSoup: (.find_all() method)

><code>doc.find_all(tag_name, attributes, limit, string, recursive=True)</code>:

* tag_name: Le nom des éléments **HTML** ex: &lt;a&gt;, &lt;tr&gt;
* attributes: un dict contenant les attributs d'une balise ex: {“class”: “abcabc”}
* limit: un nombre pour limiter le nombre de balises sous cette correspondance.
* text: une chaîne pour correspondre au contenu d'un élément 
* recursive: Par défaut, Beautiful Soup recherche tous les éléments enfants. Ainsi, le paramètre recursive = False limitera la recherche au premier élément trouvé et à son enfant uniquement.

In [36]:
tousLesDiv = doc.find_all('div')

In [37]:
len(tousLesDiv)

253

In [38]:
tousLesDivAvecClass = doc.find_all('div',{'class': 'sc-hd'})

In [29]:
len(tousLesDivAvecClass)

1

In [40]:
sixPremierDiv = doc.find_all('div', limit=6)
len(sixPremierDiv)

6

## BeautifulSoup: (.find() method)

Une autre méthode répandue est, qui fonctionne comme mais ne renvoie que la première occurrence de la balise que nous avons demandée

Dans notre recherche spécifique, nous avons des attributs dont la valeur est nulle. Donc, tout d'abord, nous organisons/analysons les données par rapport à la balise de liste d'éléments. En recherchant sur le site Web à l'aide de l'option Inspecter (clic droit sur les éléments), nous avons constaté que les informations sont disposées dans la balise div avec class = "organic-gallery-offer-outter J-offer-wrapper".

![image](images/1.png)

Le code du Web Scraping dépend de la structure de la page Web. Donc, si la conception change, votre code a également besoin d'une mise à jour. Nous utilisons une boucle for et une méthode append pour organiser les données en item_list_tags. Nous utilisons également une combinaison de [**sleep()**](https://www.programiz.com/python-programming/time/sleep, 'doc de sleep') et de [**randint**](https://www.geeksforgeeks.org/python-randint-function/, 'doc de randint') pour être doux sur le site Web. (La fonction randint() choisira un entier aléatoire entre les limites supérieures et inférieures données, dans ce cas, **10** et **2**, respectivement, pour chaque itération de la boucle. Utilisation de la fonction randint() en combinaison avec la fonction **sleep()** aidera à ajouter des pauses courtes et aléatoires dans la vitesse d'exploration du programme. La fonction sleep() arrêtera l'exécution du programme pendant le nombre de secondes donné. Le nombre de secondes sera introduit de manière aléatoire dans la fonction sleep à l'aide de randint( ). Reportez-vous au code ci-dessous pour faciliter la compréhension)

In [76]:
# Maintenant nous avons un résumé au dessus de la fonction 
def get_item_list_tags():
    item_list_tags = []
    for page in range(1,5):
        items_url = f"https://www.alibaba.com/trade/search?fsb=y&IndexArea=product_en&CatId=&SearchText=Inflight+Items&viewtype=G&tab={page}"
        response = requests.get(items_url)
        page_contents = response.text
        if response.status_code != 200:
            raise Exception('Failed to load page {}'.format(items_url))
        doc = BeautifulSoup(page_contents, "html.parser")
        for item in doc.find_all("div", {'class': "organic-gallery-offer-outter J-offer-wrapper"}):
                item_list_tags.append(item)
        sleep(randint(2,10))
        print('Téléchargement du nombre de page', page)  
    return item_list_tags
item_list_tags = get_item_list_tags()
len (item_list_tags)

Téléchargement du nombre de page 1
Téléchargement du nombre de page 2
Téléchargement du nombre de page 3
Téléchargement du nombre de page 4


32